# make korad txt file (coco format)

In [255]:
import json
import os
import glob
import random
import numpy as np
import tqdm

### VOC box (xmin,ymin,xmax,ymax) to COCO box (xmin,ymin, box-width(xmax-xmin), box-height(ymax-ymin))

In [256]:
def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = '{:.6f}'.format(round(x*dw,6))
    w = '{:.6f}'.format(round(w*dw,6))
    y = '{:.6f}'.format(round(y*dh,6))
    h = '{:.6f}'.format(round(h*dh,6))
    return x,y,w,h

### path random 으로 섞기

In [257]:
np.random.seed(2)

root_path = os.path.expanduser("~/Desktop/joo/data/korad/data/custom/")
label_set = set(map(lambda x : x[-13:-4] if x[-4:]==".jpg" else x[-14:-5],glob.glob(root_path+"images/*/*")))
labels = list(label_set)
random.shuffle(labels)

In [258]:
with open("./korad_boxes/all_list.json", "r") as f:
    all_dict = json.load(f)
    train_ratio = 0.82
    val_ratio = 0.15
    test_ratio = 0.03

    train_set = 0
    val_set = 0
    test_set = 0
    
    error_set = 0

    for index, data_id in enumerate(all_dict.keys()):
        # if index == 10 : break
            
        rd_num = np.random.random()
        
        _, _, json_true = all_dict[data_id]
        
        if json_true == None :rd_num = 1
        
        if rd_num < train_ratio :
            train_set += 1
            datatype = ["train"]

        elif rd_num < train_ratio + val_ratio:
            val_set += 1
            datatype = ["val"]
        else :
            if rd_num == 1 : error_set += 1
            test_set += 1
            datatype = ["test"]
        all_dict[data_id] = all_dict[data_id] + datatype
 

In [261]:
print(f"train_set : {train_set}개 ({train_set/23400:.2f}%)")
print(f"val_set : {val_set}개 ({val_set/23400:.2f}%)")
print(f"test_set : {test_set - error_set} + {error_set}(error file) = {test_set}개 ({test_set/23400:.2f}%)")

train_set : 18421개 (0.79%)
val_set : 3419개 (0.15%)
test_set : 646 + 914(error file) = 1560개 (0.07%)


In [262]:
classes = ["ConcreteCrack", "AlligatorCrack","Spalling", "Efflorescene", "Exposure"]
year = "2022"
save_path = ["train","val","test"]

# txt 초기화
for write_type in save_path:
    with open(root_path+write_type+year+".txt", "w") as file: file.writelines("")

# txt in labels
for index, file in enumerate(labels):
    
    number, img_file, json_file_name, data_type = all_dict[file]

    with open(root_path+data_type+year+".txt", "a") as file: 
            file.writelines("./images/"+number+"/"+img_file+"\n")

    if json_file_name == None: continue
    
    with open(root_path+"labeling/"+number+"/"+json_file_name, "r") as f: json_file = json.load(f)

    width = json_file['images']['width']
    height = json_file['images']['height']
    save_true = True

    for label in json_file['labels']:
        class_num = str(classes.index(label['label']))
        xmin = label['boundingPoly']['vertices'][0]['xmin']
        ymin = label['boundingPoly']['vertices'][0]['ymin']
        xmax = label['boundingPoly']['vertices'][0]['xmax']
        ymax = label['boundingPoly']['vertices'][0]['ymax']

        bb = convert((width,height),(xmin,xmax,ymin,ymax))
        
        if save_true:
            with open(root_path+"labels/"+data_type+year+"/"+json_file_name[:-5]+".txt", "w") as file:
                file.writelines(class_num + " "+bb[0]+" "+bb[1]+" "+bb[2]+" "+bb[3]+" \n")
            save_true = False
        else:
            with open(root_path+"labels/"+data_type+year+"/"+json_file_name[:-5]+".txt", "a") as file:
                file.writelines(class_num + " "+bb[0]+" "+bb[1]+" "+bb[2]+" "+bb[3]+" \n")
    # break

In [234]:
a,b,c_1,c_2 = 0,0,0,0
for id in all_dict:
    _,_,json_ty,ty = all_dict[id]
    if ty == "train": a += 1
    elif ty == "val": b += 1
    elif ty == "test" : 
        if json_ty != None:
            c_1 += 1
            continue
        c_2 += 1

print(a,b,c_1,c_2, c_1+c_2, a+b+c_1+c_2)

temp_set = set(map(lambda x : x[-13:-4] , glob.glob(root_path+"labels/*/*")))
print(len(label_set))
print(len(temp_set))

temp_err = label_set - temp_set
temp_dict = set()

with open(os.path.expanduser("./korad_boxes/error_list.json"), "r") as f: 
    json_file = json.load(f)
    for i in json_file.keys(): 
        temp_dict.add(i)

with open(os.path.expanduser("./korad_boxes/error_list2.json"), "r") as f: 
    json_file = json.load(f)
    for i in json_file.keys(): 
        temp_dict.add(i)        
print(len(temp_dict))

print(temp_err - temp_dict)


18412 3461 617 910 1527 23400
23400
22486
910
{'C03_02787', 'C05_03486', 'C03_01098', 'C03_00711'}
